# read_write_lib.py

In [2]:
# import cv2
import numpy as np

def image_read_ppm(dirFile):
    f = open(dirFile, 'r')
    header_1 = f.readline()
    header_2 = f.readline()
    header_3 = f.readline()

    IMG_WIDTH  = int(header_2.split()[0])
    IMG_HEIGHT = int(header_2.split()[1])

    header = np.array([header_1, header_2, header_3])
    data = f.read().replace("\r", " ").replace("\n", " ")
    dataTmp = data.split()
    dataList = dataTmp[:IMG_HEIGHT*IMG_WIDTH*3]
    outData = np.array(dataList, dtype=int).reshape(IMG_HEIGHT, IMG_WIDTH, 3)

    return outData, IMG_WIDTH, IMG_HEIGHT, header

def image_write_ppm(dirFile, imgData, header):
    header_1 = header[0]
    header_2 = header[1]
    header_3 = header[2]

    IMG_WIDTH  = int(header_2.split()[0])
    IMG_HEIGHT = int(header_2.split()[1])

    rgb = imgData.astype(np.int32)
    rgb = np.where(rgb > 255, 255, rgb)
    rgb = np.where(rgb < 0, 0, rgb)

    with open(dirFile, 'w') as f:
        f.write(header_1)
        f.write(header_2)
        f.write(header_3)
        for i in range(IMG_HEIGHT):
            for j in range(IMG_WIDTH):
                f.write(str(rgb[i,j,0]) + " " )
                f.write(str(rgb[i,j,1]) + " " )
                f.write(str(rgb[i,j,2]) + "\n")

# fixed_point_lib.py

In [3]:
import numpy as np

def func_round_diff(idata):
    odata = np.where(np.absolute(idata - idata.astype(np.int32)) >= 0.5, np.sign(idata) * (abs(idata).astype(np.int32)+1), idata.astype(np.int32))
    return odata

def round_array(idata, rl):
    odata = func_round_diff(idata/(2**rl))
    return odata

def fixed_pts_array(idata, fl):
    odata = func_round_diff(idata*(2**fl))
    return odata

# math.py

In [15]:
def cov(input_data, filter, pad=2):
    H, W = input_data.shape
    f_H, f_W = filter.shape

    output_H = (H + 2 * pad - f_H) + 1
    output_W = (W + 2 * pad - f_W) + 1

    pad_data = np.pad(input_data, [(pad, pad), (pad, pad)], 'edge')
    output = np.zeros((output_H, output_W))

    for h in range(H):
        for w in range(W):
            output[h, w] = np.sum(pad_data[h:h+f_H, w:w+f_W] * filter)
            
    return output

# def mat_mul(input_data, filter):

def yuv2rgb(input_data, y_data, filter, width, height, rl):
    yuv = input_data.copy()
    yuv[:,:,0] = y_data
    rgb = yuv.reshape(-1, 3)
    rgb = yuv.dot(filter)
    rgb = round_array(rgb, rl)
    rgb = rgb.reshape(width, height, 3)

    return rgb

# Main Code

In [71]:
import numpy as np

if __name__ == "__main__":
    #############################
    # 1. Parameter setting
    #############################
    PPM_NAME = "sample3"
    INPUT_IMAGE_DIR = "./image/input/"
    OUTPUT_IMAGE_DIR = "./image/output/"

    INPUT_IMAGE_FILE_PATH = INPUT_IMAGE_DIR + PPM_NAME + ".ppm"
    OUTPUT_IMAGE_FILE1_PAHT = OUTPUT_IMAGE_DIR + PPM_NAME + "_1.ppm"
    OUTPUT_IMAGE_FILE2_PAHT = OUTPUT_IMAGE_DIR + PPM_NAME + "_2.ppm"
    OUTPUT_IMAGE_FILE3_PAHT = OUTPUT_IMAGE_DIR + PPM_NAME + "_3.ppm"
    OUTPUT_IMAGE_FILE4_PAHT = OUTPUT_IMAGE_DIR + PPM_NAME + "_4.ppm"
    # OUTPUT_IMAGE_FILE1_PAHT = OUTPUT_IMAGE_DIR + PPM_NAME + "_adv_1.ppm"
    # OUTPUT_IMAGE_FILE2_PAHT = OUTPUT_IMAGE_DIR + PPM_NAME + "_adv_2.ppm"
    # OUTPUT_IMAGE_FILE3_PAHT = OUTPUT_IMAGE_DIR + PPM_NAME + "_adv_3.ppm"
    # OUTPUT_IMAGE_FILE4_PAHT = OUTPUT_IMAGE_DIR + PPM_NAME + "_adv_4.ppm"

    imgData, IMG_WIDTH, IMG_HEIGHT, header = image_read_ppm(INPUT_IMAGE_FILE_PATH)
    r = imgData[:,:,0]

    FILTER_SIZE = 5
    PADDING_SIZE = int(FILTER_SIZE/2)

    csc_coef = [[0.2126 , 0.7152  , 0.0722  ],
                [-0.0991, -0.33609, 0.436   ],
                [0.615  , -0.55861, -0.05639]]
    csc_coef = np.array(csc_coef)
    
    icsc_coef = [[1, 0       , 1.28033 ],
                 [1, -0.21482, -0.38059],
                 [1, 2.12798 , 0       ]]
    icsc_coef = np.array(icsc_coef)


    den_coef = np.ones((5,5)) / (FILTER_SIZE*FILTER_SIZE)
    shr_coef = -np.ones((5,5)) / (FILTER_SIZE*FILTER_SIZE - 1)
    shr_coef[2][2] = 1

    gaussian_smooting = [[1,  4,  7,  4, 1],
                         [4, 16, 24, 16, 4],
                         [7, 24, 41, 24, 7],
                         [4, 16, 24, 16, 4],
                         [1,  4,  7,  4, 1]]
    gaussian_smooting = np.array(gaussian_smooting, dtype=float)
    gaussian_smooting = gaussian_smooting/256
    laplacian_of_gaussian = [[ 0,  0, -1,  0,  0],
                             [ 0,  0, -2,  0,  0],
                             [-1, -2, 12, -2, -1],
                             [ 0,  0, -2,  0,  0],
                             [ 0,  0, -1,  0,  0]]
    laplacian_of_gaussian = np.array(laplacian_of_gaussian, dtype=float)

    csc_rl = 8
    icsc_rl = 8
    shr_rl = 8
    den_rl = 8
    gau_rl = 8
    lap_rl = 8
    
    csc_coef_fixed = fixed_pts_array(csc_coef, csc_rl)
    den_coef_fixed = fixed_pts_array(den_coef, den_rl)
    shr_coef_fixed = fixed_pts_array(shr_coef, shr_rl)
    icsc_coef_fixed = fixed_pts_array(icsc_coef, icsc_rl)
    gau_coef_fixed = fixed_pts_array(gaussian_smooting, gau_rl)
    lap_coef_fixed = fixed_pts_array(laplacian_of_gaussian, lap_rl)

    #############################
    # 2. Setting Parameter
    #############################

    mat1_coef = csc_coef_fixed
    mat2_coef = icsc_coef_fixed
    # filter1_coef = gau_coef_fixed
    # filter2_coef = lap_coef_fixed
    filter1_coef = den_coef_fixed
    filter2_coef = shr_coef_fixed

    mat1_rl = csc_rl
    mat2_rl = icsc_rl
    filter1_rl = den_rl
    filter2_rl = shr_rl
    #############################
    # 3. Main Operation
    #############################
    # Step 1. RGB to YUV
    rgb = imgData.reshape(-1, 3)
    yuv = rgb.dot(mat1_coef)
    yuv = yuv.reshape(IMG_HEIGHT, IMG_WIDTH, 3)
    yuv = round_array(yuv, mat1_rl)
    # print(yuv)

    # Step 2. Filter 1 (blur filter)
    y = yuv[:,:,0]
    y_filter1 = cov(y, filter1_coef, PADDING_SIZE)
    y_filter1 = round_array(y_filter1, filter1_rl)

    # Step 3. Filter 2 (Sharpness filter)
    y_filter2 = cov(y_filter1, filter2_coef, PADDING_SIZE)
    y_filter2 = round_array(y_filter2, filter2_rl)
    
    #############################
    # 4. Print Result
    #############################
    # Step 4. YUV to RGB
    rgb1 = yuv2rgb(yuv, yuv[:,:,0], mat2_coef, mat2_rl)
    rgb2 = yuv2rgb(yuv, y_filter1, mat2_coef, mat2_rl)
    rgb3 = yuv.copy()
    rgb3[:,:,0] = y_filter2
    rgb3[:,:,1] = y_filter2
    rgb3[:,:,2] = y_filter2
    # rgb3 = yuv2rgb(temp, y_filter2, mat2_coef, mat2_rl)
    rgb4 = yuv2rgb(yuv, y_filter1 + y_filter2, mat2_coef, mat2_rl)

    image_write_ppm(OUTPUT_IMAGE_FILE1_PAHT, rgb1, IMG_WIDTH, IMG_HEIGHT, header)
    image_write_ppm(OUTPUT_IMAGE_FILE2_PAHT, rgb2, IMG_WIDTH, IMG_HEIGHT, header)
    image_write_ppm(OUTPUT_IMAGE_FILE3_PAHT, rgb3, IMG_WIDTH, IMG_HEIGHT, header)
    image_write_ppm(OUTPUT_IMAGE_FILE4_PAHT, rgb4, IMG_WIDTH, IMG_HEIGHT, header)

    ###########################################################
    # a. verilog verification file dump
    ###########################################################
    np.savetxt('./verilog/csc_coef.txt', mat1_coef, fmt='%d')
    np.savetxt('./verilog/filter1_coef.txt', filter1_coef, fmt='%d')
    np.savetxt('./verilog/filter2_coef.txt', filter2_coef, fmt='%d')
    np.savetxt('./verilog/icsc_coef.txt', mat2_coef, fmt='%d')